In [1]:
import os
cwd_old = os.getcwd()
os.chdir('/data2/sungjaecho/data_tts')
cwd_new = os.getcwd()
print("The working directory is moved from {} to {}.".format(cwd_old, cwd_new))

The working directory is moved from /data2/sungjaecho/Projects/tacotron2/dev_ipynb to /data7/data.


In [2]:
import os
import librosa
import pandas as pd
import shutil
from tqdm.notebook import trange, tqdm

# 1. KETTS2

In [4]:
dir_database = 'KETTS2/KETTS2'

In [23]:
str_db = 'ketts2'

In [5]:
speakers = ['20m', '30f', '40m', '50f', '50m', '60f']
emo_converter = {
    'ang':'angry', 
    'dis':'disgusted', 
    'fea':'fearful', 
    'hap':'happy', 
    'neu':'neutral', 
    'sad':'sad', 
    'sur':'surprised',
}

## 1.1. Rename wav file names and move to one wav directory

In [26]:
new_dir_wav = os.path.join(dir_database, 'wav')
if not os.path.exists(new_dir_wav):
    os.mkdir(new_dir_wav)

In [28]:
cnt = 0
for speaker in speakers:
    emotion_dirs = os.listdir(os.path.join(dir_database, speaker))
    for emotion_dir in emotion_dirs:
        emotion = emo_converter[emotion_dir.split('_')[0]]
        wav_dir = os.path.join(dir_database, speaker, emotion_dir)
        wav_names = os.listdir(wav_dir)
        for wav_name in wav_names:
            wav_path = os.path.join(wav_dir, wav_name)
            script_id, wav_ext = os.path.splitext(wav_name)
            new_wav_name = '{}-{}_{}_{:03d}{}'.format(str_db, speaker, emotion, int(script_id), wav_ext)
            new_wav_path = os.path.join(new_dir_wav, new_wav_name)
            
            os.rename(wav_path, new_wav_path)
            
            if cnt < 10:
                print("Rename | {} -> {}".format(wav_path, new_wav_path))
            cnt += 1

Rename | KETTS2/KETTS2/20m/neu_500/288.wav -> KETTS2/KETTS2/wav/ketts2-20m_neutral_288.wav
Rename | KETTS2/KETTS2/20m/neu_500/335.wav -> KETTS2/KETTS2/wav/ketts2-20m_neutral_335.wav
Rename | KETTS2/KETTS2/20m/neu_500/290.wav -> KETTS2/KETTS2/wav/ketts2-20m_neutral_290.wav
Rename | KETTS2/KETTS2/20m/neu_500/308.wav -> KETTS2/KETTS2/wav/ketts2-20m_neutral_308.wav
Rename | KETTS2/KETTS2/20m/neu_500/3.wav -> KETTS2/KETTS2/wav/ketts2-20m_neutral_003.wav
Rename | KETTS2/KETTS2/20m/neu_500/191.wav -> KETTS2/KETTS2/wav/ketts2-20m_neutral_191.wav
Rename | KETTS2/KETTS2/20m/neu_500/263.wav -> KETTS2/KETTS2/wav/ketts2-20m_neutral_263.wav
Rename | KETTS2/KETTS2/20m/neu_500/81.wav -> KETTS2/KETTS2/wav/ketts2-20m_neutral_081.wav
Rename | KETTS2/KETTS2/20m/neu_500/207.wav -> KETTS2/KETTS2/wav/ketts2-20m_neutral_207.wav
Rename | KETTS2/KETTS2/20m/neu_500/256.wav -> KETTS2/KETTS2/wav/ketts2-20m_neutral_256.wav


In [32]:
for speaker in speakers:
    rm_dir_path = os.path.join(os.getcwd(), dir_database, speaker)
    print(rm_dir_path)
    shutil.rmtree(rm_dir_path)

/data7/data/KETTS2/KETTS2/20m
/data7/data/KETTS2/KETTS2/30f
/data7/data/KETTS2/KETTS2/40m
/data7/data/KETTS2/KETTS2/50f
/data7/data/KETTS2/KETTS2/50m
/data7/data/KETTS2/KETTS2/60f


## 1.2. KETTS: Make DB

In [33]:
dir_wav = os.path.join(dir_database, 'wav'); print(dir_wav)

KETTS2/KETTS2/wav


In [63]:
df_script = pd.read_csv(os.path.join(dir_database, 'KETTS2_script.csv'), encoding='utf-8')
df_script.head()

,index,script
0,1,잃어버린 서류가방을 동료가 찾아주었다.
1,2,엄마 절 낳아주셔서 감사해요.
2,3,그는 물걸레로 바닥을 계속 닦는다.
3,4,그건 정말 호랑이 입에 바쳐온 거나 같다.
4,5,그렇게 먹다가는 금방 돼지가 되고 말 걸.


In [47]:
wav_names = os.listdir(dir_wav)
wav_paths = [os.path.join(dir_wav, wav) for wav in wav_names]
print("Total #wavs:", len(wav_names))

Total #wavs: 21000


In [57]:
str_database = 'ketts2'
text_list = list()
id_list = list()
speaker_list = list()
emotion_list = list()
sex_list = list()
for wav_name, wav_path in tqdm(zip(wav_names, wav_paths), total=len(wav_paths)):
    str_id = os.path.splitext(wav_name)[0]
    speaker, emotion, script_id = str_id.split('_')
    
    id_list.append(str_id)
    speaker_list.append(speaker)
    emotion_list.append(emotion)
    if speaker[-1] == 'm':
        sex_list.append('m')
    if speaker[-1] == 'f':
        sex_list.append('w')
    
    script = df[df['index'] == int(script_id)].script.values[0]
    text_list.append(script)

In [ ]:
duration_list = list()
for i in tqdm(range(len(wav_paths))):
    y, sr = librosa.load(wav_path)
    duration = librosa.get_duration(y, sr)
    duration = round(duration, 3)
    duration_list.append(duration)

In [58]:
n_samples = len(wav_paths)
df = pd.DataFrame({
    'database':[str_database]*n_samples,
    'id':id_list,
    'wav_path':wav_paths,
    #'duration':duration_list,
    'text':text_list,
    'speaker':speaker_list,
    'sex':sex_list,
    'emotion':emotion_list,
    'lang':['ko']*n_samples,
})

In [60]:
df.shape

(21000, 8)

In [61]:
df.head()

,database,id,wav_path,text,speaker,sex,emotion,lang
0,ketts2,ketts2-40m_fearful_361,KETTS2/KETTS2/wav/ketts2-40m_fearful_361.wav,새로 익히고 있는 증권업무도 전혀 낯선 일입니다.,ketts2-40m,m,fearful,ko
1,ketts2,ketts2-50f_angry_237,KETTS2/KETTS2/wav/ketts2-50f_angry_237.wav,소원을 들어주는 요술램프가 있었으면 좋겠다.,ketts2-50f,w,angry,ko
2,ketts2,ketts2-60f_sad_277,KETTS2/KETTS2/wav/ketts2-60f_sad_277.wav,우리 앞에 어떤 가능성도 배제하지 말자.,ketts2-60f,w,sad,ko
3,ketts2,ketts2-30f_neutral_354,KETTS2/KETTS2/wav/ketts2-30f_neutral_354.wav,나는 정도를 걸으면서 양국은 하나라는 생각을 응원한다.,ketts2-30f,w,neutral,ko
4,ketts2,ketts2-40m_neutral_098,KETTS2/KETTS2/wav/ketts2-40m_neutral_098.wav,무엇이든 가장 보편적인 것이 사람들에게 사랑받는 법이다.,ketts2-40m,m,neutral,ko
